In [5]:
import pandas as pd
import random
import io
import matplotlib.pyplot as plt
import sys
import os
import codecs
import string
import re
import pymorphy2 
from collections import OrderedDict
from nltk.stem.snowball import SnowballStemmer 
from stop_words import get_stop_words
from ipywidgets import IntProgress
from IPython.display import display

In [6]:
def log_progress(sequence, every=1):
    progress = IntProgress(min=0, max=len(sequence), value=0)
    display(progress)
    
    for index, record in enumerate(sequence):
        if index % every == 0:
            progress.value = index
        yield record 

Фильтр (feature toggle) по iot категориям:

In [7]:
iot_class = True

Гиперпараметры:

In [8]:
#минимальная длина стоп слова:
stop_length = 2 

Читаем директорию с файлами каталога:

In [9]:
list_of_files = os.listdir('data/top')
files_num = len(list_of_files) 

Выбор необходимых атрибутов:

In [10]:
market_features = ['Категория', 'Производитель', 'Название модели']

Читаем файлы с необходимыми атрибутами в один фрейм:

In [11]:
df = pd.DataFrame()
for item in log_progress(list_of_files, every=1):      
    df_file = pd.read_csv(('data/top/' + str(item)), delimiter=';', encoding='windows-1251', usecols=market_features)
    df = pd.concat([df, df_file], axis=0)

IntProgress(value=0, max=313)

Переименовываем столбцы:

In [12]:
df = df.rename(columns={'Категория': 'class_m', 'Производитель': 'ven_m', 'Название модели': 'model_m'})

Число записей:

In [13]:
dim_before = int(df.shape[0])
print('Число записей:', int(df.shape[0]))

Число записей: 3421187


Сброс записей с пропущенными значениями:

In [14]:
df.dropna(how='any', axis=0, inplace=True)

Удаление пунктуации из строковых атрибутов:

In [15]:
for item in list(string.punctuation): 
    df[['class_m', 'model_m']] = df[['class_m', 'model_m']].applymap(lambda x: x.replace(item, ' '))

Убираем пробелы и приводим к нижнему регистру:

In [16]:
df = df.applymap(lambda x: re.sub(' +', ' ', x) if isinstance(x, str) else x)
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

Сброс дубликатов:

In [17]:
df.drop_duplicates(inplace=True, keep='first', subset=['class_m', 'ven_m', 'model_m'])

Преобразование типов:

In [18]:
df = df.astype({'class_m': 'str', 'ven_m': 'str', 'model_m': 'str'})

Выводим число нулей:

In [19]:
print('Число нулей для всех признаков:', dict(df.isnull().sum()))

Число нулей для всех признаков: {'class_m': 0, 'ven_m': 0, 'model_m': 0}


Записываем категории в filter/classes.txt:

In [20]:
with open('filter/classes.txt', 'w') as f:
    f.writelines('%s\n' % item for item in list(set(df['class_m'])))        

На основе файла составляем список исключений без sim. Записываем в except_class.txt. Читаем исключения из файла в exceptions:

In [21]:
exceptions = []
with io.open('filter/except_class.txt', 'r', newline=None, encoding='utf-8') as file:
    for item in file:
        item = item.replace('\n', '') 
        exceptions.append(item)

Оставляем категории товаров, которых нет в списке исключений:

In [22]:
if iot_class is True:
    df = df[~df['class_m'].isin(exceptions)] 
    print('Число записей:', int(df.shape[0]))

Число записей: 7298


Удаляем дублирующиеся слова в названии модели:

In [23]:
df['model_m'] = df['model_m'].apply(lambda x: ' '.join(OrderedDict.fromkeys(x.split())))

Удаляем производителей из названий модели:

In [24]:
for item in log_progress(list(set(df['ven_m'])), every=1):
    df['model_m'] = df['model_m'].apply(lambda x: x.replace(item, ''))

IntProgress(value=0, max=351)

Составляем список стоп слов русского словаря с фильтром по длине: 

In [25]:
stop_words_ru = [item for item in get_stop_words('russian') if len(item) > stop_length]

Удаляем стоп слова в названии модели:

In [26]:
for stop in log_progress(stop_words_ru, every=1):
    remove_stops = (lambda x: ' '.join([item for item in x.split() if item != stop]))
    df['model_m'] = df['model_m'].apply(remove_stops)    

IntProgress(value=0, max=375)

Создаем вектор ключевых слов по названиям категорий:

In [27]:
keys = " ".join(list(set(df['class_m'])))
keys = list(set(keys.split()))

Приводим слова категорий к стандартной форме:

In [28]:
morph = pymorphy2.MorphAnalyzer()
keys = [morph.parse(item)[0].normal_form for item in keys]

Выделяем основы/корни из списка - стемминг:

In [29]:
stemmer = SnowballStemmer("russian") 
keys = [stemmer.stem(item) for item in keys]

Если в слове из наименования модели присутствует 'основа' - генератор ключевого слова, то слово удаляется: 

In [30]:
for key in keys:
    remove_keys = (lambda x: ' '.join([item for item in x.split() if (key not in item)]))
    df['model_m'] = df['model_m'].apply(remove_keys)

Статистики по распределению числа слов в наименовании модели:

In [31]:
print('Среднее число слов:', round(df['model_m'].str.split().apply(len).mean()))
print('Разброс числа слов:', round(df['model_m'].str.split().apply(len).max()-df['model_m'].str.split().apply(len).min()))

Среднее число слов: 3
Разброс числа слов: 11


Статистики по распределению числа символов в наименовании модели:

In [32]:
print('Среднее число символов:', round(df['model_m'].apply(len).mean()))
print('Разброс числа символов:', round(df['model_m'].apply(len).max()-df['model_m'].apply(len).min()))

Среднее число символов: 12
Разброс числа символов: 68


Число записей:

In [33]:
dim_after = int(df.shape[0])
print('Число записей:', int(df.shape[0]))

Число записей: 7298


Компрессия по числу записей:

In [34]:
print('Компрессия по числу записей:', round(dim_before/dim_after))

Компрессия по числу записей: 469


Описание фрейма:

In [35]:
df.describe()[:2][:]

,class_m,ven_m,model_m
count,7298,7298,7298
unique,11,351,6512


Выводим случайный сэмпл:

In [36]:
df.sample(n=5)

,class_m,ven_m,model_m
14,автомагнитолы,pioneer,mvh s110uba
6042,мобильные телефоны,samsung,galaxy a5
9995,gps навигаторы,prology,imap 5300
10088,видеорегистраторы,explay,d 002
6866,мобильные телефоны,fly,iq4490i era nano 10


Записываем csv в директорию:

In [37]:
df.to_csv('data/market_clean.csv', index=False)